In [6]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

from ipywidgets import widgets
from IPython.display import display, clear_output



# Cosmological inference

## History
People have been interested in the Universe for a long time. It started small, when people were tracking planetary motions in the years BCE. The first estimate for the Earth radius happend already in 240 BCE by Eratosthenes. Although he used different distance units, common for the time (stadia), for which we don't have exact conversion rate to metric, he could estimate the Earth radius ~1-2% error compared to todays known value. 

Although the history starts long ago, the modern cosmology starts in the early 1900s. One key aspect is the Einstein's general relativity theory, which is also base for todays discoveries. But theory itself is not enough. Another very import aspect is the experiment. Einstein's theory wouldn't have been successful if Eddington wouldn't have proved it in 29th May 1919. Decade later it was proposed by Georges Lemaître and 2 years later confirmed by Edwin Hubble, that more recessional velocity of a galaxy depends on the distance to Earth. So it was the early 1900s when we started to have better theory and also more data about the Universe.

Another big discovery was done by accident in the 1960s by Americans Arno Allan Penzias and Robert Woodrow Wilson. The discovery was made with a antenna made for detecting radio waves for another project. They saw, that signal was stronger than expected and ruled out possibilities of hardware malfunction and static noise. The signal kept "noisy". Later they were put into contact Robert H. Dicke, astrophysicists 60km away whos reasearch group had a plan to start detecting CMB signals. They estimated that the signal corresponds to 3.5 K background. Later measurements have measured CMB more precisely:
* Cosmich Background Explorer (COBE) - discovery of anisotropies in the CMB, launched in 1989
* Wilkinson Microwave Anisotropy Probe (WMAP) - more accurate measurement of CMB, launched in 2001
* Planck sattellite - Latest, most accurete measurement (last data release in 2018) launched in 2009 

## Frequentist vs Bayesian

There are two approaches for statistics: frequentist vs Bayesian. The main differenece of these two approaches comes down to how they view parameters to be estimated

### **Frequentist approach**

In frequentist approach the probability is defined as 

$$P(x) = \lim_{n\rightarrow \infty} \frac{n_x}{n}\,.$$

In other words, probability is a long term frequency of an event in repeated, independent trials.

In frequentist approach the parameters of interest are fixed (our model parameters), but unknown. So the random process applies to data acquisition process. To estimate the parameters we must observed data. There are many tecniques to estimate the parameters, some more common are: method of moments, maximum likelihood estimation (MLE).

The results of an analysis are typically:
 1) Point estimates (with confidence intervals) - Example: Mean height = 170 cm, CI: 168-172cm
 2) Binary (for hypothesis testing) rejecing null hypothesis or not: p-value=0.03 < significance level = 0.05 thus we can reject null hypothesis

### **Bayesian approach**

The key concept of Bayesian approach is Bayes' theorem

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}\,.$$

Here we talk about probability as "degree of belief" based on data and our previous belief. Compared to frequentist approach, we now introduce new concept - prior, which encodes this "previous belief".

In Bayesian statistics the parameters are not fixed anymore are are treated as random variables, following some distribution. The data itself is now "fixed".

Both approaches likelihood (quantity measuring how well model explains  observed data), which will be part of the next chapter.

## Bayesian framework

### **Bayes' Theorem:**

As it's name says, Bayesian framework is based on Bayes' Theorem. Renaming some variables from the Bayes's Thereom more useful way we have now

$$P(\mathbf{\theta}|d) = \frac{P(d|\mathbf{\theta})P(\mathbf{\theta})}{P(d)}\,.$$

where $d$ - data, $\theta$  - parameters of interest. In this formulation we have

#### Likelihood **$P(d|\mathbf{\theta})$**: 

Measures how well the model parameters $\mathbf{\theta}$ are compatible with the observed data $d$. Likelihood itself is not a probability distribution and thus does not need to normalize to 1.

Common likelihood functions:
 * Gaussian (of independent measurements)
 * Poisson (Discrete distribution with independent events)
 * Binomial (Binary observations with fixed probability)

 In the case of multiple likelihoods, we can calculate the joint (total) likelihood as 
 $$\mathcal{L}_{\rm tot} = \prod_{i=1}^{n} \mathcal{L}_i$$


#### Prior **$P(\mathbf{\theta})$**:

This is unique to Bayesian inference. It's a probability distribution over the parameters reflecting our belief before seeing data. Selecting prior is very important and can change the results.

 * Informative prior
   * We have knowledge of previous experiment(s) results and/or data.
   * Can help MCMC sampling to be more efficient
   * Posterior is heavily dependent on prior
 * Weakly informative prior
   * Help to rule out impossible or extreme values
   * Helps to stabilize estimates (in case of low data scenarios)
   * Posterior is mostly dependent on data
 * Uninformative prior
   * Posterior is dominated by likelihood (data)

#### Evidence **$P(d)$**:

Otherwise known as marginal likelihood. Evidence is normalisation factor, as we require and is often ignored
$$\int P(\theta|d) d\theta = 1\,,$$
we need that 
$$\int P(d|\theta)P(\theta)d\theta = P(d)\,.$$
Sometimes it is also said that it measures how well model predicts the data. Latter makes it possible to compare different models. Using Bayes' theorem again we can now show that

$$\frac{P(M_1 | d)}{P(M_2 | d)} = \overbrace{\frac{P(D | M_1)}{P(D | M_2)}}^{\text{Bayes factor}\ K} \times \frac{P(M_1)}{P(M_2)}$$
where $M$ now represents a "model".

To interpret the Bayes' factor $K$ value we can use Jeffrey's scale which says that 
 * $K < 1 \Rightarrow$ model $M_2$ is better
 * $1 \leq K < 10^{1/2} \Rightarrow$ model difference is barely worth mentioning
 * $10^{1/2} \leq K < 1 \Rightarrow$ model $M_1$ is substantially better
 * $10 \leq K \leq 100 \Rightarrow$ model $M_1$ is strongly better
 * $100 < K \Rightarrow$ model $M_1$ is decisevely better

#### Posterior **$P(\mathbf{\theta} | d)$**:

Mostly people are interested in the posterior distribution. This shows parameters' distribution after taking into account prior and seeing the data. Usually final result of the Bayesian inference. For the posterior we can later calculate:
 * Mean, variance
 * Credible interval - smallest interval that contains N% of data.
   * For example 95% credible interval is smallest interval that contains 95% of distribution mass.



In [7]:
def gaussian_credible_interval(mean, std, credible_interval=0.95):
    assert credible_interval < 1, "Credible interval must be smaller than 1."
    plt.clf()
    clear_output(wait=True)
    f_log_gaussian = lambda _x: -(_x-mean)**2/(2*std**2)
    
    x = np.linspace(mean-5*std, mean+5*std, 10_000)
    y = f_log_gaussian(x)
    
    alpha = 1-credible_interval
    z = sc.stats.norm.ppf(1-alpha/2)
    # plt.clf()
    # plt.figure(figsize=(9, 3.5))
    plt.plot(x, np.exp(y-max(y)))
    plt.fill_between(x, np.exp(y-max(y)), alpha=0.6)
    plt.plot([mean, mean], [0, 1], label="Mean", color="red")
    plt.plot([mean-std, mean-std], [0, np.exp(f_log_gaussian(mean-std)-max(y))], color="green")
    plt.plot([mean+std, mean+std], [0, np.exp(f_log_gaussian(mean+std)-max(y))], color="green", label=f"$\\mu\\pm\\sigma$ (CI 68.27%)")
    plt.plot([mean-z*std, mean-z*std], [0, np.exp(f_log_gaussian(mean-z*std)-max(y))], color="black", linestyle="--")
    plt.plot([mean+z*std, mean+z*std], [0, np.exp(f_log_gaussian(mean+z*std)-max(y))], color="black", linestyle="--", label=f"Credible interval {credible_interval*100:.2f}%")
    plt.grid()
    plt.legend()
    plt.xlabel("x")
    plt.ylabel(r"$\propto p(x)$")

In [8]:
mean_slider = widgets.FloatText(value=1, min=0, max=5, step=0.1, description='Mean')
std_slider = widgets.FloatText(value=1, min=0, max=5, step=0.1, description='Std')
credible_interval_slider = widgets.BoundedFloatText(value=0.6827, min=0, max=0.9999, step=0.05, description='CI')
display(widgets.interact(gaussian_credible_interval, mean=mean_slider, std=std_slider, credible_interval=credible_interval_slider))


interactive(children=(FloatText(value=1.0, description='Mean', step=0.1), FloatText(value=1.0, description='St…

<function __main__.gaussian_credible_interval(mean, std, credible_interval=0.95)>

### Example - Coin flip

* What to look for?
  * Strong infomative prior (change $\alpha$ or $\beta$ higher. The bigger the value the "stronger" the prior.)
  * Non-informative prior ($\alpha=\beta=1$ is uniform prior)
  * The more data we have
    * the more posterior is influenced by the likelihood
    * likelihood gets narrower

In [9]:
def binomial_distribution(theta, heads, tails):
    return theta**heads*(1-theta)**tails

def beta_distribution(theta, alpha, beta):
    return theta**(alpha-1)*(1-theta)**(beta-1)

def log_binomial_distribution(theta, heads, tails):
    return heads*np.log(theta) + tails*np.log(1-theta)

def log_beta_distribution(theta, alpha, beta):
    return (alpha-1)*np.log(theta) + np.log(1-theta)*(beta-1)

def plot_coin_bayes(n_heads, n_tails, alpha, beta, use_log=False):
    """ Plot distribution functions (posterior, prior, likelihood) for coin flips, given number of heads and tails.
    
    Comments:
     * alpha = beta = 1 => Uniform prior distribution  
     * alpha > beta => Distribution more tilted to lower values, "left"
     * alpha < beta => Distribution more tilted to higher values, "right"

    Args:
        n_heads (int): number of heads
        n_tails (int): number of tails
        alpha (float): Beta distribution parameter 1 for prior
        beta (float): Beta distribution parameter 2 for prior
        use_log (bool): If use logarithm of pdfs to calculate prior. likelihood and posterior
    """
    if (alpha+beta > 750) or (n_heads+n_tails > 750):
        print("It's advised to use_log=True")
    
    theta = np.linspace(1e-8, 1-1e-8, 1000)

    n_total = n_heads + n_tails
    mean_prior = (alpha)/(alpha+beta)
    mean_likelihood = n_heads/(n_total)
    mean_posterior = (alpha + n_heads)/(n_total + alpha + beta)


    if use_log:
        log_dist_prior = log_beta_distribution(theta, alpha, beta)
        log_dist_likelihood = log_binomial_distribution(theta, n_heads, n_tails)
        log_dist_posterior = log_dist_prior+log_dist_likelihood
        
        dist_prior = np.exp(log_dist_prior-max(log_dist_prior))
        dist_likelihood = np.exp(log_dist_likelihood-max(log_dist_likelihood))
        dist_posterior = np.exp(log_dist_posterior-max(log_dist_posterior))
        f_mean_prior = np.exp(log_beta_distribution(mean_prior, alpha, beta) - max(log_dist_prior))
        f_mean_posterior = np.exp(log_beta_distribution(mean_posterior, alpha + n_heads, beta + n_tails) - max(log_dist_posterior))

    else:
        dist_prior = beta_distribution(theta, alpha, beta)
        dist_likelihood = binomial_distribution(theta, n_heads, n_tails)
        dist_posterior = dist_prior*dist_likelihood
        f_mean_prior = beta_distribution(mean_prior, alpha, beta)/max(dist_prior)
        f_mean_posterior = beta_distribution(mean_posterior, alpha + n_heads, beta + n_tails)/max(dist_posterior)

    with plt.rc_context({"font.size": 13}):
        fig = plt.figure(figsize=(13, 8))
        plt.style.use("seaborn-v0_8-dark")

        # First row, first column
        ax1 = plt.subplot2grid((2, 2), (0, 0))
        ax1.plot([mean_prior, mean_prior], [0, f_mean_prior], color="black", linestyle="--", label=f"$\\mathbb{{E}}[p(\\theta)] = {mean_prior:.2f}$")
        ax1.plot(theta, dist_prior/max(dist_prior), color="green", label="Prior")
        ax1.fill_between(theta, dist_prior/max(dist_prior), color="lightgreen", alpha=0.6)
        ax1.set_title("Prior")

        # First row, second column
        ax2 = plt.subplot2grid((2, 2), (0, 1))
        ax2.plot([mean_likelihood, mean_likelihood], [0, 1], color="black", linestyle="--", label=f"$\\mathbb{{E}}[p(d|\\theta)] = {mean_likelihood:.2f}$")
        ax2.plot(theta, dist_likelihood/max(dist_likelihood), color="blue", label="Likelihood")
        ax2.fill_between(theta, dist_likelihood/max(dist_likelihood), color="lightblue", alpha=0.6)
        ax2.set_title("Likelihood")

        # Second row, span both columns
        ax3 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
        ax3.plot([mean_posterior, mean_posterior], [0, f_mean_posterior], color="black", linestyle="--", label=f"$\\mathbb{{E}}[p(\\theta|d)] = {mean_posterior:.2f}$")
        ax3.plot(theta, dist_posterior/max(dist_posterior), color="red", label="Posterior")

        ax3.plot(theta, dist_prior/max(dist_prior), color="green", label="Prior", linestyle="--")
        ax3.plot(theta, dist_likelihood/max(dist_likelihood), color="blue", label="Likelihood", linestyle="--")

        ax3.fill_between(theta, dist_posterior/max(dist_posterior), color="salmon", alpha=0.6)
        ax3.set_title("Posterior")
        
        for _ax in [ax1, ax2, ax3]:
            _ax.legend(frameon=True)
            _ax.grid()
            _ax.set_xlim(0, 1)
            _ax.set_xlabel(r"$\text{p(Head)}$")

In [10]:
heads_slider = widgets.IntText(value=1, min=0, max=1000, step=0.1, description='N heads')
tails_slider = widgets.IntText(value=1, min=0, max=1000, step=0.1, description='N tails')
alpha_slider = widgets.BoundedFloatText(value=1, min=1, max=1000, step=1, description=r'$\alpha$')
beta_slider = widgets.BoundedFloatText(value=1, min=1, max=1000, step=1, description=r'$\beta$')
use_log_checkbox = widgets.Checkbox(value=False, description='Use log porbabilities')
display(widgets.interact(plot_coin_bayes, n_heads=heads_slider, n_tails=tails_slider, alpha=alpha_slider, beta=beta_slider, use_log=use_log_checkbox))


interactive(children=(IntText(value=1, description='N heads', step=0), IntText(value=1, description='N tails',…

<function __main__.plot_coin_bayes(n_heads, n_tails, alpha, beta, use_log=False)>

### **Marginalization**

Marginalization is the process of summing/integrating out variables that we are not interested in.

We had posterior 
$P(\theta, D) = P(\theta_1, \theta_2, \dots | D)\,.$
Let's say, we are not interested in the effects cause by nuisance parameters $\theta_3$ and $\theta_4$, which we have to use when calculating, but are not relevant to our theoretical model. This happens for example, if the parameters describe the measurement process itself. In Bayesian statistics it is possible to easily remove them from analysis by doing

$$P(\theta_1, \theta_2, \theta_5, \dots | D) = \int \int P(\theta_1, \theta_2, \dots | D) d\theta_3 d\theta_4\,.$$

This is usually done numerically and when we use results produced by MCMC it's even simpler.


## Cosmological datasets

* Run 1 [Purely late-time] - Pantheon+ (without SH0ES calibration) + DESI DR2;
* Run 2 [Purely early-time] - CMB-SPA;
* Run 3 [Combined scenario] - CMB-SPA + Pantheon+ (without SH0ES calibration) + DESI DR2;
* Run 4 [Purely late-time] - Pantheon+ (with SH0ES calibration) + DESI DR2;
* Run 5 [Combined scenario] - CMB-SPA + Pantheon+ (with SH0ES calibration) + DESI DR2

### **CMB** - Early Universe
 * Encodes information about early universe
 * Independent parameters
   * At the decoupling:
     * $\Omega_b$ - Baryion density parameter
     * $\Omega_m$ - Matter density parameter
     * $A_s$ - amplitude of the scalar power spectrum
     * $n_s$ - scalar spectral index
   * Post-decoupling:
     * $\theta_{MC}$ - angular scale of the sound horizon (depends on $H_0$)
     * $\tau$ - reionization optical depth
   * Model depentent (a few examples): 
     * $w$ - equation of state (eos)
     * $m_{\nu_i}$ - neutrino masses
   * In principle different parameters can be selected as independent ($\theta_{MC} \rightarrow \Omega_0, \Omega_{\Lambda0}$)
   
[More about CMB parameters from a lecture by Syksy Räsänen ](https://www.mv.helsinki.fi/home/syrasane/cosmo/lect2024_10.pdf)

#### Planck 2018
 * Main goal was to measure CMB
 * Up to date most accurate CMB power spectrum measurement

#### BICEP/Keck
 * Focus on detecting B-mode polarization in CMB 
   * Signature of primordial gravitational waves from inflation

<figure  style="text-align: center;">
  <img src="images/CMB.jpg" alt="CMB" style="width: 75%; height: auto;">
  <figcaption>Picture of a CMB</figcaption>
</figure>


### **Supernova Measurements** - Late universe

Type Ia Supernovaes are mainly from late universe ($z\leq1$). We measure apparent magnitude which depends on the luminosity distance

$$D_L = (1+z)\frac{c}{H_0} \int \limits_{0}^{z} \frac{dz'}{E(z')}\,,\ E(z) = \sqrt{\Omega_m(1+z)^3 + \Omega_\Lambda}\,.$$
For the moment we assume flat universe. Supernovae measurement by itself can't measure $H_0$ directly. If distances to Supernovaes are added we can also measure local $H_0$.

**NB! Supernovae data and CMB measurements give different values for H_0! $\Rightarrow$ Hubble tension**

<figure  style="text-align: center;">
  <img src="images/HubbleTension.png" alt="Hubble tension" style="width: 75%; height: auto;">
  <figcaption>Picture of a Hubble tension <a href=https://arxiv.org/pdf/1907.10625>https://arxiv.org/pdf/1907.10625 </a></figcaption>
</figure>


Some of the most known datasets:
 * DES (Dark Energy Survey)
   * ~1800 Sn type Ia with z~0.02 to ~1
   * [Cosmology results](https://arxiv.org/pdf/2401.02929) and [newer analysis](https://arxiv.org/pdf/2511.07517)
   * Data: [Older dataset from Cobaya](https://github.com/CobayaSampler/sn_data/tree/master/DESY5), [Newer analysis method Cobaya](https://github.com/CobayaSampler/sn_data/tree/master/DES-Dovekie)
 * Pantheon+
   * ~1500 Sn type Ia with z~0.03 to 2.3 
   * [Dataset article](https://arxiv.org/pdf/2112.03863) and [Cosmology results](https://arxiv.org/pdf/2202.04077)
   * Data: [GitHub Cobaya](https://github.com/CobayaSampler/sn_data/tree/master/PantheonPlus)
 * SH0ES (Supernova $H_0$ for the Equation of State)
   * Started 2005 
   * Main aim is to measure $H_0$ (local)
   * Measures Cepheid variable stars and Type Ia supernovae
   * Use distance ladder to measure distances
   * [Article](https://arxiv.org/pdf/2112.04510)

### **Baryon Acoustic Oscillations (BAO)** - Late universe

Experiments measuring large scale structure. Acts as a "standard" ruler. 

BAO was created near decoupling when electrons and photons were coupled. In this time gravity tried to pull the matter together but the radiation pressure pushed it apart. This created sound waves inside the plasma. After cooling the sound waves "stopped" and left imprints into the current matter distribution. From the theory we can calculate the sound horizon (how far the sound waves moved) and using results from Placnk (to get $\omega_m$ and $\omega_r$) we get that $$r_s \approx 147\ \rm Mpc\,,$$ which is also seen in the matter power spectrum measurments. 

<figure  style="text-align: center;">
  <img src="images/BAO_creation.png" alt="Hubble tension" style="width: 75%; height: auto;">
  <figcaption>Picture from: Daniel J. Eisenstein (Arizona U., Astron. Dept. - Steward Observ.), Hee-jong Seo (Arizona U., Astron. Dept. - Steward Observ.), Martin J. White (UC, Berkeley, Astron. Dept.), "On the Robustness of the Acoustic Scale in the Low-Redshift Clustering of Matter" <a href=https://arxiv.org/abs/astro-ph/0604361>https://arxiv.org/abs/astro-ph/0604361</a> </figcaption>
</figure>
More about [BAO](https://arxiv.org/pdf/0910.5224).

* DESI (Dark Energy Spectroscopic Instrument)
  * Data: [DESI Data documentation](https://data.desi.lbl.gov/doc/) and also from [GitHub Cobaya](https://github.com/CobayaSampler/bao_data/tree/master/desi_bao_dr2)